# Import libraries and configure settings

In [0]:
!pip install imagenet-c
!apt-get install libmagickwand-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmagickwand-dev is already the newest version (8:6.9.7.4+dfsg-16ubuntu6.8).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import make_grid
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
from IPython import display
from tqdm import tqdm_notebook as tqdm
import pickle
import os
import math
import gc
import cv2
from imagenet_c import corrupt
from google.colab import drive

# Connect to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Mount Google Drive
drive.mount('/content/drive')

# Prepare for importing modules
!pip install import-ipynb
import import_ipynb
# Install PyDrive wrapper and import libraries
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Initialize models

In [0]:
# Import init_non_IBE module and variables
init_non_IBE = drive.CreateFile({'id':'1Fg9_nuYr2dHV8oSQrTDZJsnf2vltS-DA'})
init_non_IBE.GetContentFile('init_non_IBE.ipynb')
from init_non_IBE import net, criterion

net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Import init_IBE module and variables
init_IBE = drive.CreateFile({'id':'1k4JdlYX0ANxVEFDBxxp_pcqBzc2p_u2F'})
init_IBE.GetContentFile('init_IBE.ipynb')
from init_IBE import net_IBE, attention, criterion_IBE

net_IBE.to(device)
if device == 'cuda':
    net_IBE = torch.nn.DataParallel(net_IBE)
    cudnn.benchmark = True

importing Jupyter notebook from init_non_IBE.ipynb
importing Jupyter notebook from init_IBE.ipynb


# Prepare dataset and corrupt images

In [0]:
# Import prepare_dataset module and variables
prepare_dataset = drive.CreateFile({'id':'15l7vAAf7L4sVRUuRbLwIedrN3vhwQUfb'})
prepare_dataset.GetContentFile('prepare_dataset.ipynb')
from prepare_dataset import testset, testloader, num_classes, num_subset, aug_total, transform_no_aug

corruptions_for_test = [-1, 0, 3, 7, 9, 10, 11, 2, 5, 6, 14, 17, 18]
corruptions_for_test_names = ['No Distortion', 'Gaussian Noise', 'Defocus Blur', 'Snow', 'Fog', 'Brightness', 'Contrast', 'Impulse Noise', 'Motion Blur', 'Zoom Blur', 'JPEG Compression', 'Spatter', 'Saturate']
corruptions_for_test_files = ['no_distortion', 'gaussian_noise', 'defocus_blur', 'snow', 'fog', 'brightness', 'contrast', 'impulse_noise', 'motion_blur', 'zoom_blur', 'jpeg_compression', 'spatter', 'saturate']

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
gc.enable()
plt.ioff()
np.seterr(divide='ignore', invalid='ignore')

def corrupt_images(corrupt_type, corrupt_level):
    corrupted_set = []
    for i in range(len(testset)):
        img = testset.data[i]
        img = cv2.resize(img, dsize=(224,224), interpolation=cv2.INTER_CUBIC)   # Upsample CIFAR-10 images
        corrupted_img = corrupt(img, severity=corrupt_level, corruption_number=corrupt_type)
        corrupted_img = cv2.resize(corrupted_img, dsize=(32,32), interpolation=cv2.INTER_CUBIC)    # Restore to original image size
        corrupted_img = transform_no_aug(corrupted_img)
        corrupted_set.append([corrupted_img, testset.targets[i]])
    return corrupted_set

importing Jupyter notebook from prepare_dataset.ipynb
Files already downloaded and verified
Files already downloaded and verified


# Test and save results

In [0]:
### Specify model to test ('IBE' | 'non-IBE') ###
crt_model = 'non-IBE'

def load_model(aug_idx, set_idx):
    crt_state = torch.load('checkpoint_{}/aug_{}/ckpt_{}.pth'.format(crt_model, aug_idx, set_idx))
    if crt_model == 'IBE':
        crt_net = net_IBE
        crt_criterion = criterion_IBE
    else:
        crt_net = net
        crt_criterion = criterion
    crt_net.load_state_dict(crt_state['net'])
    return crt_net, crt_criterion

def place_bin(correct, prob):
    for k in np.arange(10):
        s = (((0.0 + k*0.1) < prob) & (prob <= (0.1 + k*0.1))) * 1
        idx = (s == 1).nonzero()
        c = (correct * 1) & s
        bin_size[k] += s.sum().item()
        bin_conf_sum[k] += prob[idx].sum().item()
        bin_correct[k] += c.sum().item()

def visualize(x, seg_out, corruption, level, aug_idx, set_idx, acc, attn):
    hue = (torch.argmax(seg_out, dim=1).float() + 0.5)/10
    x -= x.min()
    x /= x.max()
    gs_im = x.mean(1)
    gs_mean = gs_im.mean()
    gs_min = gs_im.min()
    gs_max = torch.max((gs_im-gs_min))
    gs_im = (gs_im - gs_min)/gs_max
    hsv_im = torch.stack((hue.float(), attn.squeeze().float(), gs_im.float()), -1)
    im = hsv_to_rgb(hsv_im.cpu().detach().numpy())
    ex = make_grid(torch.tensor(im).permute(0,3,1,2), normalize=True, nrow=25)
    attns = make_grid(attn, normalize=False, nrow=25)
    attns = attns.cpu().detach()
    inputs = make_grid(x, normalize=True, nrow=25).cpu().detach()
    fig = plt.figure(figsize=(20,8))
    plt.imshow(np.concatenate((inputs.numpy().transpose(1,2,0),ex.numpy().transpose(1,2,0), attns.numpy().transpose(1,2,0)), axis=0))
    fig.suptitle('CIFAR10 Aug:{}, Set:{}, Test:{:.3f}'.format(aug_idx, set_idx, acc), fontsize=20)
    vis_path = 'visualizations/{}_{}/aug_{}'.format(corruption, level, aug_idx)
    if not os.path.isdir(vis_path):
        os.makedirs(vis_path)
    fig.savefig('{}/{}.png'.format(vis_path, set_idx))

In [0]:
def test_on_model(crt_net, crt_criterion, dataloader, corruption, level, aug_idx, set_idx):
    confidence_batch = np.zeros(len(dataloader))
    global bin_size
    global bin_correct
    global bin_conf_sum
    bin_size = np.zeros(10)
    bin_correct = np.zeros(10)
    bin_conf_sum = np.zeros(10)

    crt_net.eval()
    epoch_loss = 0.0
    correct = 0
    total = 0
    test_pbar = tqdm(dataloader)
    with torch.no_grad():
        for i, (x, _label) in enumerate(test_pbar):
            x = x.cuda()
            _label = _label.cuda()

            if crt_model == 'IBE':
                label = F.one_hot(_label, num_classes=10).float()
                seg_out = net_IBE(x)
                attn = attention(seg_out)
                logit = torch.log(torch.exp(seg_out*0.5).mean((-2,-1)))*2
            else:
                label = _label
                logit = crt_net(x)
            
            loss = crt_criterion(logit, label)
            epoch_loss += loss.item()
            total += _label.size(0)
            _, predicted = logit.max(1)
            correct += predicted.eq(_label).sum().item()
            test_pbar.set_description('Acc: {:.3f}%'.format(100.*correct/total))

            # Save testing confidence
            sm = nn.Softmax(dim=1)
            probs = sm(logit)
            max_prob, _ = probs.max(1)
            confidence_batch[i] = torch.mean(max_prob)

            # Store data for each bin
            correct_mask = (logit.argmax(-1)==_label)
            place_bin(correct_mask, max_prob)
    
    accuracy_all[set_idx] = correct/total
    confidence_all[set_idx] = np.mean(confidence_batch)
    binned_accuracy_all.append(bin_correct/bin_size)
    binned_confidence_all.append(bin_conf_sum/bin_size)
    bin_size_all.append(bin_size)
    if crt_model == 'IBE':
        visualize(x, seg_out, corruption, level, aug_idx, set_idx, 100.*correct/total, attn)

for corrupt_num in corruptions_for_test:
    # Account for no corruption case
    if corrupt_num == -1:
        level_total = 1
    else:
        level_total = 5

    for level in range(1,level_total+1):
        if corrupt_num == -1:
            print('Running: Original Test Set (No Distortion)')
            corrupted_testloader = testloader
            idx = 0
        else:
            idx = corruptions_for_test.index(corrupt_num)
            print('Running: {} Level {}'.format(corruptions_for_test_names[idx], level))

            # Corrupt data
            corrupted_testset = corrupt_images(corrupt_num, level)
            corrupted_testloader = torch.utils.data.DataLoader(corrupted_testset, batch_size=100, shuffle=False, num_workers=2)

        # Test and obtain results
        for aug in range(0, aug_total):
            print('-> Aug: {}'.format(aug))
            accuracy_all = np.zeros(num_subset)
            confidence_all = np.zeros(num_subset)
            binned_accuracy_all = []
            binned_confidence_all = []
            bin_size_all = []
            for subset in range(0, num_subset):
                print('----> Subset: {}'.format(subset))
                crt_net, crt_criterion = load_model(aug, subset)
                test_on_model(crt_net, crt_criterion, corrupted_testloader, corruptions_for_test_files[idx], level, aug, subset)
            
            # Save results
            var_path = 'vars_{}/{}_{}'.format(crt_model, corruptions_for_test_files[idx], level)
            if not os.path.isdir(var_path):
                os.makedirs(var_path)
            with open('{}/aug_{}.pkl'.format(var_path, aug), 'wb') as f:
                pickle.dump([accuracy_all, confidence_all, binned_accuracy_all, binned_confidence_all, bin_size_all], f)

-> Aug: 0
----> Subset: 0



----> Subset: 1



----> Subset: 2



----> Subset: 3



----> Subset: 4


...


----> Subset: 7



----> Subset: 8



----> Subset: 9



Running: Zoom Blur Level 1
-> Aug: 0
----> Subset: 0


Buffered data was truncated after reaching the output size limit.